In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'/home/sagemaker-user/Text_Summarizer_HuggingFace'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path 
    data_path: Path 
    model_path: Path 
    tokenizer_path: Path 
    metric_file_name: Path 



In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories


In [6]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name,
        )

        return model_evaluation_config




In [9]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import evaluate
from datasets import load_from_disk
import torch
from tqdm import tqdm
import pandas as pd


In [1]:
import evaluate

from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_inputs(self,list_of_elements,batch_size):
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]

    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,batch_size=16,device="cuda" if torch.cuda.is_available() else "cpu",column_text="article",column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_inputs(dataset[column_text],batch_size))
        target_batches = list(self.generate_batch_sized_inputs(dataset[column_summary],batch_size))

        for article_batch,target_batch in tqdm(zip(article_batches,target_batches),total=len(dataset)//batch_size):
            inputs = tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")

            summaries = model.generate(input_ids=inputs['input_ids'].to(device),attention_mask=inputs['attention_mask'].to(device),length_penalty=0.8,num_beams=8,max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]

            decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries,references=target_batch)
            score = metric.compute()
            return score

            
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
        rouge_metric = evaluate.load("rouge")

        # rouge_metric = rouge_metric

        score = self.calculate_metric_on_test_ds(dataset_samsum_pt['test'][0:10],rouge_metric,model_pegasus,tokenizer,batch_size=2,column_text='dialogue',column_summary='summary')
        rouge_dict = {rn: score[rn] for rn in rouge_names}
        df = pd.DataFrame(rouge_dict.items(), columns=['Metric', 'Score'])
        df.to_csv(self.config.metric_file_name, index=False)





/home/sagemaker-user/Text_Summarizer_HuggingFace/myvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'ModelEvaluationConfig' is not defined

In [14]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation = ModelEvaluation(config=model_evaluation_config)
model_evaluation.evaluate()

[2025-03-12 00:22:03,041: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-12 00:22:03,059: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-12 00:22:03,064: INFO: common: created directory at: artifacts]
[2025-03-12 00:22:03,082: INFO: common: created directory at: artifacts/model_evaluation]


NameError: name 'rougue_metric' is not defined